In [2]:
# data table
import pandas as pd
import os
# map

os.environ['USE_PYGEOS'] = '0'

import geopandas as gpd


datadismap=gpd.read_file(os.path.join('maps','Provincias.zip'))

datadismap.head()

,id,c_ut,DEPARTAMEN,PROVINCIA,geometry
0,0,0313,Cochabamba,Mizque,"POLYGON ((-65.61829 -17.67455, -65.59538 -17.6..."
1,1,0701,Santa Cruz,Andrés Ibáñez,"POLYGON ((-62.97381 -18.02444, -63.08615 -18.0..."
2,2,0409,Oruro,Sabaya,"POLYGON ((-68.47194 -19.35205, -68.47718 -19.3..."
3,3,0501,Potosí,Tomás Frías,"POLYGON ((-65.87315 -18.97139, -65.86752 -18.9..."
4,4,0404,Oruro,Sajama,"POLYGON ((-68.47914 -18.61758, -68.48010 -18.6..."
